# Packages

In [ ]:
# Matplotlib setup
from matplotlib_setup import mpl_setup

# Modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as font_manager
import seaborn as sns

# Mute warnings
import warnings
warnings.filterwarnings("ignore")

# Setup

In [ ]:
# directories
wd = '/Users/lneumann/Documents/'
database_dir = wd + 'Products/ALMOND/dense_gas_letter/database/'
fits_dir = wd + 'Products/ALMOND/dense_gas_letter/fits/'
plot_dir = wd + 'Products/ALMOND/dense_gas_letter/plots/appendix/'

# SNR threshold for masking
SNR_threshold = 3
scatter_sigma = 3 # used to compute the scatter (set higher to eliminate contribution from noise)

# markers
marker_almond = 'h'
marker_empire = 'o'

# colors
scatter_alpha = 0.3
scatter_color = 'grey'
color_almond = 'tab:blue'
color_empire = 'tab:red'

# constants to compute surface densities and dense gas fraction
alpha_co10 = 4.35  # [M_Sun/pc^2/(K km/s)] (Bolatto et al. 2013)

alpha_HCN = 15  # [M_Sun/pc^2/(K km/s)] (Schinnerer & Leroy 2024)
c_dense = alpha_HCN/alpha_co10

# alpha CO prescription
aco = 'SL24'
# aco = 'MW'

# labels
xlabel_list = [r'$\Sigma_{\star}$ [M$_{\odot}\,\mathrm{pc}^{-2}$]',
               r'$\Sigma_{\rm mol}$  [M$_{\odot}\,\mathrm{pc}^{-2}$]',
               r'$P_{\rm DE}/k_{\rm B}$ [$\mathrm{K}\,\mathrm{cm}^{-3}$]']
ylabel_list = [r'$\frac{W_{\mathrm{HCN}(1-0)}\;[\mathrm{K}\,\mathrm{km}\,\mathrm{s}^{-1}]}{W_{\mathrm{CO}(1-0)}\;[\mathrm{K}\,\mathrm{km}\,\mathrm{s}^{-1}]}$',
               r'$\frac{\Sigma_\mathrm{SFR}\;[\mathrm{M}_{\odot}\,\mathrm{yr}^{-1}\,\mathrm{pc}^{-2}]}{W_{\mathrm{HCN}(1-0)}\;[\mathrm{K}\,\mathrm{km}\,\mathrm{s}^{-1}]}$']

# data files
xlabel_list_file = ['Sd_star', 'Sd_mol_'+aco, 'P_DE_'+aco]
ylabel_list_file = ['hcn_co21', 'sfr_hcn']
xdata_label_list = ['SD_star [Msun/pc2]', 'SD_mol (%s) [Msun/pc2]' % aco, 'P_DE (%s) [kB.K/cm3]' % aco]

# fitting file
xlabel_fitting_list = ['Sd_star', 'Sd_mol_'+aco, 'P_DE_'+aco]
ylabel_fitting_list = ['hcn_co', 'sfr_hcn']

# number of galaxies in panel
# ngalaxies_list = ['31 galaxies\n' + r'@ $\sim 1-2\,$kpc', '31 galaxies\n' + r'@ $\sim 1-2\,$kpc', '26 galaxies\n' + r'@ $\sim 1-2\,$kpc']
ngalaxies_list = ['31 galaxies', '31 galaxies', '26 galaxies']

# secondary x-axis for CO from Sigma_mol
def Sdmol_to_Wco(Sd_mol):
    return Sd_mol / alpha_co10
def Wco_to_Sdmol(W_co21):
    return W_co21 * alpha_co10

# secondary y-axis for SFE from SFR/HCN
def sfr_hcn_ratio_to_sfedense(sfr_hcn_ratio):
    return sfr_hcn_ratio / alpha_HCN  # units: Myr^-1
def sfedense_to_sfr_hcn_ratio(sfedense):
    return sfedense * alpha_HCN

# secondary y-axis for fdense from HCN/CO
def Wratio_to_fdense(Wratio):
    return Wratio * c_dense
def fdense_to_Wratio(fdense):
    return fdense / c_dense

# box for labels
box = dict(boxstyle='round', facecolor='white', edgecolor='k', alpha=0.8)  # set box

# HCN/CO, SFR/HCN vs Molecular Cloud Properties (ALMOND+EMPIRE)

In [ ]:
savepng = False
savepdf = True

# show density contours
density_contours = True

# figure name
savepath = plot_dir + 'HCN_scaling_relations_empire_vs_almond'

######################################

# plotting settings
xlim_list = [(0.9*3e1, 6e3), (0.9*5e0, 4e2), (0.9*1e4, 5e6)]
ylim_list = [(5e-3, 1.5e-1), (2e-8, 8e-7)]

######################################

# matplotlib setup
mpl_setup(figtype='paper-1/1', ms=4)

# create figure
fig = plt.figure()
fig.subplots_adjust(hspace=0, wspace=0)

######################################

# get number of rows and columns
nrows = len(ylabel_list)
ncols = len(xlabel_list)

# loop over subplots
for i in range(nrows*ncols):
    
    # make subplot
    ax = fig.add_subplot(nrows, ncols, i+1)
    
    # get column and row indeces
    id_row = i // ncols
    id_col = i % ncols

    # labels
    xlabel_file = xlabel_list_file[id_col]
    ylabel_file = ylabel_list_file[id_row]
    xdata_label = xdata_label_list[id_col]
    xlabel_fitting = xlabel_fitting_list[id_col]
    ylabel_fitting = ylabel_fitting_list[id_row]
        
    # data range
    xlim = xlim_list[id_col]
    ylim = ylim_list[id_row]

    # make data arrays
    x, x_err =  np.array([]), np.array([])
    y, y_err = np.array([]), np.array([])
    y_ul, y_ll = np.array([]), np.array([])
        
    ########## ALMOND STACKS ############

    # data file
    fname_table = database_dir + 'almond_and_empire_database_stacks_%s.csv' % xlabel_file
    df = pd.read_csv(fname_table)
    df = df[df['Survey']=='ALMOND']
    
    # get data
    xdata = df[xdata_label]
    xdata_err = df['e_' + xdata_label]
    hcn = df['W_HCN [K.km/s]']
    hcn_err = df['e_W_HCN [K.km/s]']
    co = df['W_CO10 [K.km/s]']
    co_err = df['e_W_CO10 [K.km/s]']
    sfr = df['SD_SFR [Msun/yr/kpc2] (no inclination correction)'] * 1e-6  # Msun/yr/pc2
    sfr_err = df['e_SD_SFR [Msun/yr/kpc2] (no inclination correction)'] * 1e-6 # Msun/yr/pc2
    if ylabel_file == 'hcn_co21':
        ydata = hcn/co
        ydata_err = np.abs(ydata) * np.sqrt((hcn_err/hcn)**2 + (co_err/co)**2)
    elif ylabel_file == 'sfr_hcn':
        ydata = sfr/hcn
        ydata_err = np.abs(ydata) * np.sqrt((hcn_err/hcn)**2 + (sfr_err/sfr)**2)

    # compute limits
    snr = ydata/ydata_err
    mask = (snr >= SNR_threshold)
    
    ydata_ul = np.full_like(ydata, np.nan)
    ydata_ll = np.full_like(ydata, np.nan)
    
    if ylabel_file == 'hcn_co21':
        ydata_ul[~mask] = hcn_err[~mask] * SNR_threshold / co[~mask]
            
    elif ylabel_file == 'sfr_hcn':
        ydata_ll[~mask] = sfr[~mask] / (hcn_err[~mask] * SNR_threshold)

    # append to array
    x = np.concatenate((x, xdata))
    x_err = np.concatenate((x_err, xdata_err))
    y = np.concatenate((y, ydata))
    y_err = np.concatenate((y_err, ydata_err))
    y_ul = np.concatenate((y_ul, ydata_ul))
    y_ll = np.concatenate((y_ll, ydata_ll))
        
    ########## PLOTTING ############
    # plot binned data
    ax.errorbar(xdata[mask], ydata[mask], yerr=ydata_err[mask], marker=marker_almond, mfc=color_almond, mec='k', mew=0.5, 
                ecolor=color_almond, ls='none', elinewidth=0.5, capsize=0, label='ALMOND', zorder=5)
    
    # plot upper limits
    if ylabel_file == 'hcn_co21':
        (_, caps, _) = ax.errorbar(xdata[~mask], ydata_ul[~mask], yerr=ydata_ul[~mask]/10, uplims=True, linestyle='none',
                                   marker='None', elinewidth=0.5, color=color_almond, zorder=4)
    # plot lower limits
    elif ylabel_file == 'sfr_hcn':
        (_, caps, _) = ax.errorbar(xdata[~mask], ydata_ll[~mask], yerr=ydata_ll[~mask]/6, lolims=True, linestyle='none',
                                   marker='None', elinewidth=0.5, color=color_almond, zorder=4)
    for cap in caps:
        cap.set_color(color_almond)
        cap.set_ms(3)
        cap.set_mew(0)
    
    # density contours
    if density_contours:
        sns.kdeplot(x=xdata[mask], y=ydata[mask], log_scale=True, levels=[0.25, 0.5, 0.75, 1], color=color_almond, fill=True, alpha=0.5, zorder=1, ax=ax)
        sns.kdeplot(x=xdata[mask], y=ydata[mask], log_scale=True, levels=[0.25, 0.5, 0.75, 1], color=color_almond, zorder=2, ax=ax)
    
    ########## EMPIRE STACKS ############

    # data file
    fname_table = database_dir + 'almond_and_empire_database_stacks_%s.csv' % xlabel_file
    df = pd.read_csv(fname_table)
    df = df[df['Survey']=='EMPIRE']
    df = df[df['Galaxy']!='ngc0628']
    df = df[df['Galaxy']!='ngc2903']
    df = df[df['Galaxy']!='ngc4321']
    
    # get data
    xdata = df[xdata_label]
    xdata_err = df['e_' + xdata_label]
    hcn = df['W_HCN [K.km/s]']
    hcn_err = df['e_W_HCN [K.km/s]']
    co = df['W_CO10 [K.km/s]']
    co_err = df['e_W_CO10 [K.km/s]']
    sfr = df['SD_SFR [Msun/yr/kpc2] (no inclination correction)'] * 1e-6  # Msun/yr/pc2
    sfr_err = df['e_SD_SFR [Msun/yr/kpc2] (no inclination correction)'] * 1e-6 # Msun/yr/pc2
    if ylabel_file == 'hcn_co21':
        ydata = hcn/co
        ydata_err = np.abs(ydata) * np.sqrt((hcn_err/hcn)**2 + (co_err/co)**2)
    elif ylabel_file == 'sfr_hcn':
        ydata = sfr/hcn
        ydata_err = np.abs(ydata) * np.sqrt((hcn_err/hcn)**2 + (sfr_err/sfr)**2)

    # compute limits
    snr = ydata/ydata_err
    mask = (snr >= SNR_threshold)
    
    ydata_ul = np.full_like(ydata, np.nan)
    ydata_ll = np.full_like(ydata, np.nan)
    
    if ylabel_file == 'hcn_co21':
        ydata_ul[~mask] = hcn_err[~mask] * SNR_threshold / co[~mask]
            
    elif ylabel_file == 'sfr_hcn':
        ydata_ll[~mask] = sfr[~mask] / (hcn_err[~mask] * SNR_threshold)

    # append to array
    x = np.concatenate((x, xdata))
    x_err = np.concatenate((x_err, xdata_err))
    y = np.concatenate((y, ydata))
    y_err = np.concatenate((y_err, ydata_err))
    y_ul = np.concatenate((y_ul, ydata_ul))
    y_ll = np.concatenate((y_ll, ydata_ll))
        
    ########## PLOTTING ############
    
    # plot binned data
    ax.errorbar(xdata[mask], ydata[mask], yerr=ydata_err[mask], marker=marker_empire, mfc=color_empire, mec='k', mew=0.5, 
                ecolor=color_empire, ls='none', elinewidth=0.5, capsize=0, label='EMPIRE', zorder=5)

    # plot upper limits
    if ylabel_file == 'hcn_co21':
        (_, caps, _) = ax.errorbar(xdata[~mask], ydata_ul[~mask], yerr=ydata_ul[~mask]/10, uplims=True, linestyle='none',
                                   marker='None', elinewidth=0.5, color=color_empire, zorder=4)
    # plot lower limits
    elif ylabel_file == 'sfr_hcn':
        (_, caps, _) = ax.errorbar(xdata[~mask], ydata_ll[~mask], yerr=ydata_ll[~mask]/6, lolims=True, linestyle='none',
                                      marker='None', elinewidth=0.5, color=color_empire, zorder=4)
    for cap in caps:
        cap.set_color(color_empire)
        cap.set_ms(3)
        cap.set_mew(0)

    # density contours
    if density_contours:
        sns.kdeplot(x=xdata[mask], y=ydata[mask], log_scale=True, levels=[0.25, 0.5, 0.75, 1], color=color_empire, fill=True, alpha=0.5, zorder=1, ax=ax)
        sns.kdeplot(x=xdata[mask], y=ydata[mask], log_scale=True, levels=[0.25, 0.5, 0.75, 1], color=color_empire, zorder=2, ax=ax)
        

    ########## FITTING ##############

    # load fit results
    fname_fitting = fits_dir + 'almond_and_empire_fits_' + ylabel_fitting + '_vs_' + xlabel_fitting + '.csv'
    df = pd.read_csv(fname_fitting)
    intercept = df['intercept'][0]
    slope = df['slope'][0]
    x_off = df['x-axis offset'][0]
    y_scatter = df['scatter (residuals)'][0]
    
    # plot linmix median line fit
    x_fit = np.logspace(np.log10(xlim[0]), np.log10(xlim[1]), 10)
    y_fit = 10**(intercept + slope * (np.log10(x_fit)-x_off))
    ax.plot(x_fit, y_fit, ls='solid', c='k', lw=2, zorder=1)

    # plot scatter
    y_sca_up = 10**(intercept + slope * (np.log10(x_fit)-x_off) - y_scatter)
    y_sca_dw = 10**(intercept + slope * (np.log10(x_fit)-x_off) + y_scatter)
    ax.fill_between(x_fit, y_sca_up, y_sca_dw, color=scatter_color, lw=0, alpha=scatter_alpha)
    ax.plot(x_fit, y_sca_up, ls='dashed', color='k', lw=1, zorder=1)
    ax.plot(x_fit, y_sca_dw, ls='dashed', color='k', lw=1, zorder=1)

    ######################################
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    # ax.grid(ls='dotted')
    ax.set_xscale('log')
    ax.set_yscale('log')

    ############ secondary axis ############
     
    if (id_row == 0) & (id_col == (ncols-1)):
        secyax = ax.secondary_yaxis('right', functions=(Wratio_to_fdense, fdense_to_Wratio)) # secondary y-axis for f_dense
        secyax.set_ylabel(r'$f_\mathrm{dense}$')
        # secyax.tick_params(which='both', direction='out')
        ax.tick_params(which='both', right=False)
        
    if (id_row == 1) & (id_col == (ncols-1)):
        secyax = ax.secondary_yaxis('right', functions=(sfr_hcn_ratio_to_sfedense, sfedense_to_sfr_hcn_ratio)) # secondary y-axis for SFE
        secyax.set_ylabel(r'$\mathrm{SFE}_\mathrm{dense}\;[\mathrm{yr}^{-1}]$')
        # secyax.tick_params(which='both', direction='out')
        ax.tick_params(which='both', right=False)

    ############ axis labels ############

    # x-axis labels
    if id_row == (nrows-1):
        ax.set_xlabel(xlabel_list[id_col])
    else:
        ax.tick_params(labelbottom=False)
        ax.set_xlabel('')

    # y-axis labels
    if id_col == 0:
        ax.set_ylabel(ylabel_list[id_row], size='x-large')
    else:
        ax.tick_params(labelleft=False)  
        ax.set_ylabel('')    
        
    ############ annotations ############
    if id_row == 0:
        mpl.rcParams['text.usetex'] = False
        font = font_manager.FontProperties(family='Arial', style='normal', size=6)
        ax.text(0.75, 0.1, ngalaxies_list[id_col], ha='center', va='center', transform=ax.transAxes, fontproperties=font)
        mpl.rcParams['text.usetex'] = True
    
    
    ############ legend ############
    
    if (id_row == 0) & (id_col == 0):
        mpl.rcParams['text.usetex'] = False
        font = font_manager.FontProperties(family='Arial', style='normal', size=8)
        legend1 = ax.legend(loc='upper left', prop=font)
        mpl.rcParams['text.usetex'] = True

################################

# save plot
if savepng:
    plt.savefig(savepath + '.png')
if savepdf:
    plt.savefig(savepath + '.pdf')

plt.show()